In [1]:
import pandas as pd
import os
from os import path
import pickle
import json
import numpy as np
from tqdm import tqdm
import math

In [8]:
df = pd.read_csv("../../Dataset/datasetBinary.csv")

In [9]:
train_cols = df.columns.difference(['customer_types', 'conversion', 'treat'])
X = df[train_cols] 
y = df['treat']

In [10]:
df.head()

,Unnamed: 0,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion,customer_types,treat
0,0,10,142.44,1,0,1,0,2,1,0,2,1
1,1,6,329.08,1,1,3,1,1,0,0,0,0
2,2,7,180.65,0,1,1,1,1,1,0,2,1
3,3,9,675.83,1,0,3,1,1,-1,0,2,1
4,4,2,45.34,1,0,2,0,1,1,0,2,1


In [11]:
targetCol = "treat"

In [12]:
#undersampling imports
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import CondensedNearestNeighbour
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import EditedNearestNeighbours

In [13]:
#other imports
#data partitioning
from sklearn.model_selection import train_test_split, cross_val_score

#classification
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA


In [14]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.25, random_state=12, stratify=y)


In [15]:
from collections import Counter

In [16]:
Counter(y)

Counter({1: 42694, 0: 21306})

In [17]:
from utilsBalancing import makeAndSaveToFileDecisionTreeResults

base dataset classification

In [18]:
clfs, results = makeAndSaveToFileDecisionTreeResults (X_train, y_train, X_test, y_test, "resampled/based")

unbalanced classes above

In [19]:
rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))

Resampled dataset shape Counter({0: 15979, 1: 15979})


In [20]:
clfs, results = makeAndSaveToFileDecisionTreeResults (X_res, y_res, X_test, y_test, "resampled/RandomUnderSampler")

In [21]:
from sklearn.cluster import MiniBatchKMeans
from imblearn.under_sampling import ClusterCentroids

In [22]:
cc = ClusterCentroids(
    estimator=MiniBatchKMeans(
        n_init=1, random_state=42
    ), 
    random_state=42
)
X_res, y_res = cc.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))

In [ ]:
clfs, results = makeAndSaveToFileDecisionTreeResults (X_res, y_res, X_test, y_test, "resampled/ClusterCentroids")

In [ ]:
from imblearn.over_sampling import SMOTE


In [ ]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))

Resampled dataset shape Counter({2: 28449, 0: 28449, 3: 28449, 1: 28449})


In [ ]:
clfs, results = makeAndSaveToFileDecisionTreeResults (X_res, y_res, X_test, y_test, "resampled/smote")


In [26]:
# Concatenate the features and target into a single DataFrame
resampled_df = pd.concat([X_res, y_res], axis=1)

# Save the DataFrame to a CSV file
resampled_df.to_csv('resampled_dataset.csv', index=False)